In [1]:
from __future__ import print_function
from __future__ import division

from backbone_cnn.customize_vgg16 import vgg16_customize
from torchsummary import summary
from torch.autograd import Variable
from torch.optim import lr_scheduler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import time
import math
import os
import copy
import shutil
import matplotlib
import seaborn as sns
from collections import OrderedDict
print("Pytorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)
%matplotlib inline

Pytorch Version:  1.1.0
Torchvision Version:  0.2.2


In [ ]:
input_size = 224

data_dir = './data/Car'

num_classes = 1000

batch_size = 4

num_epochs = 100

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
scratch_model = vgg16_customize(pretrained=False)

scratch_model.num_classes = num_classes

scratch_model.fc3 = nn.Linear(scratch_model.fc3.in_features, num_classes)

summary(scratch_model.to(device), input_size=(3, 224, 224))

In [ ]:
print(scratch_model)

In [ ]:
# # Save model as bin file:
# for param_tensor in scratch_model.state_dict():
#     torch.save(scratch_model.state_dict()[param_tensor], './checkpoint_bin/{}.bin'.format(param_tensor))

In [ ]:
# Create a figure for distribution histogram
fig, axe = plt.subplots(16,2, figsize=(50,300))
A = [] 
B = []
for param in scratch_model.state_dict():
    a = torch.load('./checkpoint_bin/{}.bin'.format(param))
    A.append(a.to('cpu').numpy().flatten())
    B.append(param)
i = 0
for r in range(16):
    for c in range(2):
        ax=axe[r,c].set_title("{}".format(B[i]), fontsize=48)
        sns.distplot(A[i], kde=False, bins=100, ax=axe[r,c], color='green')
        i = i + 1

In [ ]:
## Save image
A = []
B = []
for param in scratch_model.state_dict():
    a = torch.load('./checkpoint_bin/{}.bin'.format(param))
    A.append(a.to('cpu').numpy().flatten())
    B.append(param)

for i in range(32):
    plt.title("{}".format(B[i]), fontsize = 16)
    sns.distplot(A[i], kde=False, bins=100, color='green')
    plt.annotate('Maximum: ' + str(np.round(np.max(A[i]),2)), (10, 10), xycoords='axes pixels')
    plt.annotate('Minimum: ' + str(np.round(np.min(A[i]),2)), (10, 40), xycoords='axes pixels')
    plt.annotate('Average: ' + str(np.round(np.mean(A[i]),2)), (10, 70), xycoords='axes pixels')
    plt.annotate('Variance: ' + str(np.round(np.std(A[i]),2)), (10, 100), xycoords='axes pixels')
    plt.savefig('./distribution_histogram/'+str(B[i]) + '.png')
    plt.close()

In [ ]:
TEST = 'test'
TRAIN = 'train'
VAL = 'val'

data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
    ])
}

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL, TEST]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=batch_size,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, VAL, TEST]
}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

for x in [TRAIN, VAL, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
class_names = image_datasets[TRAIN].classes
print(image_datasets[TRAIN].classes)

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    # plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes = next(iter(dataloaders[TRAIN]))
show_databatch(inputs, classes)

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    
    # Set model for evaluation
    model.train(False)
    model.eval() 
    
    images_so_far = 0

    for i, data in enumerate(dataloaders[TEST]):
        inputs, labels = data
        size = inputs.size()[0]
        
        inputs, labels = Variable(inputs.to(device), volatile=True), Variable(labels.to(device), volatile=True)
        
        outputs = model(inputs)
        
        _, preds = torch.max(outputs.data, 1)
        predicted_labels = [preds[j] for j in range(inputs.size()[0])]
        
        print("Ground truth:")
        show_databatch(inputs.data.cpu(), labels.data.cpu())
        print("Prediction:")
        show_databatch(inputs.data.cpu(), predicted_labels)
        
        del inputs, labels, outputs, preds, predicted_labels
        torch.cuda.empty_cache()
        
        images_so_far += size
        if images_so_far >= num_images:
            break
        
    model.train(mode=was_training) # Revert model back to original training state

In [ ]:
def eval_model(model, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(dataloaders[TEST])
    print("Evaluating model")
    print('-' * 10)
    
    for i, data in enumerate(dataloaders[TEST]):
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)

        model.train(False)
        model.eval()
        inputs, labels = data

        inputs, labels = Variable(inputs.to(device), volatile=True), Variable(labels.to(device), volatile=True)

        outputs = model(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.item() * inputs.size(0)
        acc_test += torch.sum(preds == labels.data)

        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
        
    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)

In [ ]:
scratch_model = scratch_model.to(device)
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
print("Test before training")
eval_model(scratch_model, criterion)

In [ ]:
visualize_model(scratch_model) #test before training

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg.train(True)
        
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches), end='', flush=True)
                
            # Use half training dataset
            if i >= train_batches:
                break
                
            inputs, labels = data
            
            inputs, labels = Variable(inputs.to(device)), Variable(labels.to(device))
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item() * inputs.size(0)
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train / dataset_sizes[TRAIN]
        avg_acc = acc_train / dataset_sizes[TRAIN]
        
        model.train(False)
        model.eval()
            
        for i, data in enumerate(dataloaders[VAL]):
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
                
            inputs, labels = data
            
            inputs, labels = Variable(inputs.to(device), volatile=True), Variable(labels.to(device), volatile=True)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.item() * inputs.size(0)
            acc_val += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / dataset_sizes[VAL]
        avg_acc_val = acc_val / dataset_sizes[VAL]
        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), './checkpoint/{}.pth'.format('vgg_custom_model'))
            for param_tensor in model.state_dict():
                torch.save(model.state_dict()['param_tensor'], './checkpoint_bin/{}.bin'.format(param_tensor))
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
scratch_hist = train_model(scratch_model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=num_epochs)

In [ ]:
eval_model(scratch_model, criterion)

In [ ]:
visualize_model(scratch_model, num_images=32)